In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.data.external import *
from ipywidgets import *
import pandas as pd
import numpy as np
import math
import sqlite3
from random import randint

In [60]:
# Local stuff
from api.helpers import *
import api.tab_model as tm
import api.imageclassifier as ic
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
img_learn = ic.ImageClassifier('models/v0.4/resnet101-f7-fp16-h1.0.pkl')

In [8]:
tab_learn = tm.TabModel('dbs/fungid.sqlite')

In [25]:
file = "dbs/training/training-data-v0-4.csv"
data = pd.read_csv(file)
data = data.loc[data.imgid.notna()].copy()
data['img'] = 'dbs/images/224/' + data.gbifid.astype(str) + '-' + data.imgid.astype(int).astype(str) + '.png'

In [13]:
def get_bounding_box(lat, lon, dist):
    latdiff = (180 / math.pi) * (dist / 6378137)
    londiff = (180 / math.pi) * (dist / 6378137) / math.cos(lat)
    return (lat - latdiff, lon - londiff), (lat + latdiff, lon + londiff)

def get_db_species(conn, observation, dist):
    p1, p2 = get_bounding_box(tab_item.decimallatitude, tab_item.decimallongitude, dist)
    print(p1, p2)
    cursor = conn.execute("""
    SELECT species, COUNT(*)
    FROM validobservations v 
    JOIN trainingspecies t ON v.specieskey = t.specieskey
    WHERE decimallatitude BETWEEN ? AND ? 
    AND decimallongitude BETWEEN ? AND ? 
    GROUP BY 1 ORDER BY 2;""",
                 (p1[0], p2[0], p1[1], p2[1]))
    results = cursor.fetchall()
    print(len(results))
    
def get_tab_data(data):
    tab_columns = ['kg', 'elu_class1', 'elu_class2', 'elu_class3', 'decimallatitude', 'decimallongitude', 'species', 'normalizedmonth', 'season']
    return data[tab_columns].copy()

def get_results(learner, data):
    row, clas, probs = learner.predict(data)
    return probs
                 


In [19]:
id = 1647265
tab_item = data.iloc[id]

# lh = tab_learn.get_predictions(tab_item, vocab_df)
# print(lh)

In [76]:

id = 164726
item = data.iloc[id]
obs = obs_from_series(item)

img_preds = img_learn.get_predictions(obs.image)
tab_preds = tab_learn.get_predictions(obs)

type(img_preds.img_prob * tab_preds.likelihood)

pandas.core.series.Series

In [68]:
def compare(probs1, probs2, vals2, num):
    values1, indices1 = torch.topk(probs1, num)
    idx2 = probs2.argsort(descending=True).argsort()
    values1 = values1.cpu().detach().numpy()
    probs2 = probs2.cpu().detach().numpy()
    return pd.DataFrame(
        zip(
            [img_learn.dls.vocab[idx] for idx in indices1], 
            values1, 
            [idx2[i].item() for i in indices1], 
            [probs2[i] for i in indices1],
            [vals2[i].item() for i in indices1]
        )
    )

# 124978
# 2119839 - Bad
# 1292929 - Negative nubmers = 
# 454673 - Dropped to 10 from 1
# 1068383 - Artificial fucks with stuff. 
# 193834 & 2073568- Totall throws the data into mayhem 
# 1370890 - Ignore Surface Water in ELU?
# Need to only pull 4-5 images per obs. 

id = randint(0, len(img_data))
print(id)
item = data.iloc[id]
print(item)

obs = obs_from_series(item)

tab_preds = tab_learn.get_predictions(obs)
print((tab_preds.max(), tab_preds.min(), tab_preds.loc[tab_item.species]))
img_preds = img_learn.get_predictions(obs.image)
compare(img_preds.img_prob, img_preds.img_prob * tab_preds.likelihood, lh_tensor, 50)
# img_probs.argsort(descending=True)

192201
gbifid                                   3070681339
_family                             Psathyrellaceae
genus                                   Coprinopsis
species                    Coprinopsis atramentaria
familykey                                    4184.0
genuskey                                    2534316
specieskey                                  5242740
eventyear                                      2021
eventmonth                                        3
eventday                                         29
eventdate                       2021-03-29 18:11:07
decimallatitude                           39.183645
decimallongitude                         -77.233742
kg                                               25
elu                                             222
elu_class1                 Artificial or Urban Area
elu_class2                                      NaN
elu_class3                                      NaN
normalizedmonth                                   4
seaso

In [274]:
df = pd.DataFrame(img_learn.dls.vocab, columns=["species"]).set_index("species")
df["img_prob"] = img_probs.numpy()
df.img_prob

species
Abortiporus biennis      -0.899631
Acanthophysium oakesii   -3.042892
Acarospora fuscata       -4.228813
Acarospora moenium       -1.611503
Acarospora sinopica      -4.377919
                            ...   
Xylodon flaviporus       -2.750198
Xylodon radula           -0.861849
Xylodon sambuci          -1.909232
Xylodon subtropicus      -2.787384
Xylopsora friesii        -3.136505
Name: img_prob, Length: 2957, dtype: float32

In [191]:
lh.loc['Clitocybe rivulosa']

likelihood    4.0
Name: Clitocybe rivulosa, dtype: float64

In [84]:
len(img_learn.dls.vocab), len(tab_learn.dls.vocab)

(2957, 2957)

In [34]:
tab_probs.min()

tensor(3.0625e-07)

In [125]:
url = 'https://inaturalist-open-data.s3.amazonaws.com/photos/3897840/large.jpg'
filename = 'tmp/tmp.jpg'
resized = 'tmp/tmp-resized.jpg'
resize = Resize(224, ResizeMethod.Pad, pad_mode='zeros')
download_url(url, filename)
img = PILImage.create(filename)
resize(img).save(resized)
test_probs = get_results(img_learn, resized)
print_top50(test_probs)

[(TensorBase(11.1817), TensorBase(883), 'Entoloma hochstetteri'), (TensorBase(10.8582), TensorBase(902), 'Entoloma virescens'), (TensorBase(3.0313), TensorBase(864), 'Entocybe nitida'), (TensorBase(2.9586), TensorBase(876), 'Entoloma euchroum'), (TensorBase(2.6364), TensorBase(1766), 'Mycena interrupta'), (TensorBase(2.4956), TensorBase(415), 'Chlorociboria aeruginascens'), (TensorBase(2.2694), TensorBase(2713), 'Terana coerulea'), (TensorBase(1.8844), TensorBase(157), 'Arrhenia chlorocyanea'), (TensorBase(1.7535), TensorBase(329), 'Caloscypha fulgens'), (TensorBase(1.0795), TensorBase(521), 'Clavogaster virescens'), (TensorBase(0.8473), TensorBase(2612), 'Sparassis crispa'), (TensorBase(0.7782), TensorBase(770), 'Cyanoboletus pulverulentus'), (TensorBase(0.7490), TensorBase(2423), 'Rhodocollybia maculata'), (TensorBase(0.4456), TensorBase(1744), 'Mycena amicta'), (TensorBase(0.3630), TensorBase(300), 'Byssocorticium atrovirens'), (TensorBase(0.2923), TensorBase(2627), 'Stereocaulon co